# Sudoku Solver

_____________________________________________________________________________________________

This is program will solve any input sudoku.
Mark empty spaces with 0.
_____________________________________________________________________________________________

***Constraint Sadisfaction Problem for Sudoku*** 
<br>
variables = { v_ij | 0 =< i < 9  and 0 =< j < 9 }
<br>
domains = {1,2,3,4,5,6,7,8,9}
<br>
constraints = {< (v_ij), for_all(y): ((0 =< y < 9) and (j != y)) => (v_ij != v_iy) >,
<br>
               < (v_ij), for_all(x): ((0 =< x < 9) and (j != x)) => (v_uj != v_xj) >,
<br>
               < (v_ij, for_all(v_xy): ((floor(x/3) = floor(i/3) and floor(y/3)=floor(j/3) and (x!=i or y!=j)) => v_ij!=v_xy >}
                                   


In [29]:
def getSudoku():
    return [[2,0,0, 8,0,4, 0,0,6],
          [0,0,6, 0,0,0, 5,0,0],
          [0,0,0, 0,0,0, 9,2,0],

          [3,0,0, 0,4,0, 0,0,7],
          [0,0,0, 3,0,5, 0,0,0],
          [4,0,0, 0,6,0, 0,0,9],

          [0,1,9, 0,0,0, 7,4,0],
          [0,0,8, 0,0,0, 2,0,0],
          [5,0,0, 6,0,7, 0,0,2]]

In [30]:
sudoku = getSudoku()

In [2]:
#Prints n whitespaces (for debug)
def wsPrinter(n):
    for i in range(0,n):
        print("|",end='')

In [3]:
#prints sudoku
def printSudoku(s):
    for l in s:
        print(l,"\n")

In [4]:
#find remaining values for one variables at the beginning

def findSet(sdk,x,y):
    #determine posible solutions in set
    s = []
    if(sdk[x][y]!=0):
        s = []
    else:    
        
        s = [1,2,3,4,5,6,7,8,9]
        for n in range(1,10):
            deleted = False
            # < (v_ij), for_all(x): ((0 =< x < 9) and (j != x)) => (v_uj != v_xj) >
            for i in range(0,9):
                if(n==sdk[i][y]):
                    del s[s.index(n)]
                    deleted = True
                    break 
            # < (v_ij), for_all(y): ((0 =< y < 9) and (j != y)) => (v_ij != v_iy) >
            if(not deleted):
                for j in range(0,9):
                    if(n==sdk[x][j]):
                        del s[s.index(n)]
                        deleted = True
                        break
            # < (v_ij, for_all(v_xy): ((floor(x/3) = floor(i/3) and floor(y/3)=floor(j/3) and (x!=i or y!=j)) => v_ij!=v_xy >
            if(not deleted):
                for i in range(int(x/3)*3,int(x/3)*3+3):
                    for j in range(int(y/3)*3,int(y/3)*3+3):
                        if(n==sdk[i][j]):
                            del s[s.index(n)]
                            deleted = True
                            break
    
    return s       

In [5]:
#find remaining values for all variables at the beginning

def createSets(sdk):
    sets = [[0 for a in range(0,9)] for b in range(0,9)]
    for i in range(0,9):
        for j in range(0,9):
            sets[i][j]=findSet(sdk,i,j)
            
    return sets   


In [6]:
#find variable with minimum remaining values

def findMinimum(sets):
    minI = -1
    minJ = -1
    minL = 9
    for i in range(len(sets)):
        for j in range(len(sets[i])):
            if(len(sets[i][j]) > 0 and len(sets[i][j]) < minL):
                minI = i
                minJ = j
                minL = len(sets[i][j])
                
    return [minI,minJ]

In [7]:
#find value that eliminates the least choices for other variables

def findLeastConstrainingValue(sets,x,y):
    count =[0 for n in range(0,len(sets[x][y]))]
    for i in range(0,9):
        #iterate values
        for n in range(0,len(sets[x][y])):
            #row
            if(sets[x][y][n] in sets[x][i]):
                count[n]+=1
            #column
            if(sets[x][y][n] in sets[i][y]):
                count[n]+=1
            #cell block
            icb = int(x/3)*3 + int(i/3)
            jcb = int(y/3)*3 + int(i%3) 
            if(sets[x][y][n] in sets[icb][jcb]):
                count[n]+=1
            
    return sets[x][y][count.index(min(count))]

In [8]:
#tests if sets is arc consistent if (x,y) has the value v
#(x,y) is arc consistent with (i,j) if for every value in (x,y) there is a diffrent value in (i,j)

def isArcConsistentFor(sets,x,y,v):
    arccon = True
    for i in range(0,9):
        #row
        if(i != y and len(sets[x][i])==1 and sets[x][i][0] == v):
            arccon = False
        #column
        if(i != x and len(sets[i][y])==1 and sets[i][y][0] == v):
            arccon = False
        #cell block
        icb = int(x/3)*3 + int(i/3)
        jcb = int(y/3)*3 + int(i%3) 
        if(icb!=x and jcb!=y and len(sets[icb][jcb])==1 and sets[icb][jcb][0] == v):
            arccon = False
        
    return arccon

In [9]:
# makes sets arc consistent

def makeArcConsistent(sets):
    for i in range(0,9):
        for j in range(0,9):
            for v in sets[i][j]:
                if(not isArcConsistentFor(sets,i,j,v)):
                    del sets[i][j][sets[i][j].index(v)]

In [10]:
#checks if sudoku is solved

def isSolved(sdk):
    for row in sdk:
        for v in row:
            if(v == 0):
                return False
    return True

In [31]:
def hasPossibilitysLeft(sets):
    for row in sets:
        for v in row:
            if(len(v)>0):
                return True
    return False

In [23]:
def solve(sdk):
    sdkTemp = sdk
    sets = createSets(sdk)
    while(not isSolved(sdk)):
        makeArcConsistent(sets)
        
            

In [27]:
sudoku = getSudoku()
solve(sudoku)
printSudoku(sudoku)   

[2, 0, 0, 8, 0, 4, 0, 0, 6] 

[0, 0, 6, 0, 0, 0, 5, 0, 0] 

[0, 0, 0, 0, 0, 0, 9, 2, 0] 

[3, 0, 0, 0, 4, 0, 0, 0, 7] 

[0, 0, 0, 3, 0, 5, 0, 0, 0] 

[4, 0, 0, 0, 6, 0, 0, 0, 9] 

[6, 1, 9, 0, 0, 0, 7, 4, 0] 

[7, 0, 8, 0, 0, 0, 2, 0, 0] 

[5, 0, 0, 6, 0, 7, 0, 0, 2] 



In [28]:
sets = createSets(sudoku)
makeArcConsistent(sets)
printSudoku(sets)

print(isSolved(sudoku))

[[], [3, 5, 7, 9], [1, 3, 5, 7], [], [1, 3, 5, 7, 9], [], [1, 3], [1, 3, 7], []] 

[[1, 8, 9], [3, 4, 7, 8, 9], [], [1, 2, 7, 9], [1, 2, 3, 7, 9], [1, 2, 3, 9], [], [1, 3, 7, 8], [1, 3, 4, 8]] 

[[1, 8], [3, 4, 5, 7, 8], [1, 3, 4, 5, 7], [1, 5, 7], [1, 3, 5, 7], [1, 3, 6], [], [], [1, 3, 4, 8]] 

[[], [2, 5, 6, 8, 9], [1, 2, 5], [1, 2, 9], [], [1, 2, 8, 9], [1, 6, 8], [1, 5, 6, 8], []] 

[[1, 8, 9], [2, 6, 7, 8, 9], [1, 2, 7], [], [1, 2, 7, 8, 9], [], [1, 4, 6, 8], [1, 6, 8], [1, 4, 8]] 

[[], [2, 5, 7, 8], [1, 2, 5, 7], [1, 2, 7], [], [1, 2, 8], [1, 3, 8], [1, 3, 5, 8], []] 

[[], [], [], [2, 5], [2, 3, 5, 8], [2, 3, 8], [], [], [3, 5, 8]] 

[[], [3, 4], [], [1, 4, 5, 9], [1, 3, 5, 9], [1, 3, 9], [], [1, 3, 5, 6, 9], [1, 3, 5]] 

[[], [3, 4], [3, 4], [], [1, 3, 8, 9], [], [1, 3, 8], [1, 3, 8, 9], []] 

False


In [2]:
min([5,2,3])

2